In [1]:
import pandas
import re, json
import csv

import torch
import torch.nn as nn
from datasets import load_metric,Dataset,DatasetDict, load_dataset, Sequence, Value
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, BartForConditionalGeneration
from transformers import AutoTokenizer, Trainer

import evaluate

import numpy as np
import nltk
import os
import random
from sklearn.model_selection import train_test_split
from typing import List, Optional, Tuple, Union, Dict, Any
from jointbart_step1 import myBartForConditionalGeneration
from jointbart_lmhead_step2 import myBartForConditionalGeneration
from hg_utils import GenerationMixin

In [2]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
_numpy_rng = np.random.default_rng(seed)
random.seed(seed)
np.random.seed(seed)
torch.use_deterministic_algorithms(False)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def convert_to_iob(d):
    for i in range(len(d)):
        for j in range(len(d[i])):
            if d[i][j] != 'O':
                d[i][j] = 'B-' + d[i][j]
    
    return d

In [5]:
model_checkpoint = "facebook/bart-large"
metric = evaluate.load("rouge")
f1_metric = evaluate.load("f1")

In [6]:
max_input_length = 256
max_target_length = 128

In [7]:
model = myBartForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

Some weights of myBartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
for name, param in model.named_parameters():
    if name == 'classifier.weight' or name == 'classifier.bias':
        continue
    param.requires_grad=False

In [9]:
dataset = load_dataset('pvisnrt/mod_capstone')
id2label =  {0: 'C', 1: 'M', 2: 'N', 3: 'O', 4: 'OB', 5: 'W'}
label2id = {'C': 0, 'M': 1, 'N': 2, 'O': 3, 'OB': 4, 'W': 5}

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'summary_target', 'tags'],
        num_rows: 80
    })
    validation: Dataset({
        features: ['source', 'summary_target', 'tags'],
        num_rows: 10
    })
    test: Dataset({
        features: ['source', 'summary_target', 'tags'],
        num_rows: 10
    })
})

In [11]:
tokenizer.eos_token_id 

2

In [12]:
dataset['train'] = dataset['train'].cast_column("tags", Sequence(Value("int64")))
dataset['validation'] = dataset['validation'].cast_column("tags", Sequence(Value("int64")))
dataset['test'] = dataset['test'].cast_column("tags", Sequence(Value("int64")))

In [13]:
dataset['train'].features

{'source': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'summary_target': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'tags': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [14]:
def tokenize_and_align_labels(examples):
    inputs = [doc for doc in examples['source']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, is_split_into_words=True, return_tensors='pt', padding=True)

    with tokenizer.as_target_tokenizer():
        tokenized_inputs = tokenizer(examples["summary_target"], truncation=True, is_split_into_words=True, return_tensors='pt', padding=True)

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)# Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    model_inputs['labels'] = tokenized_inputs['input_ids']

    model_inputs["decoder_tags"] = labels
    
    return model_inputs

In [15]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

In [16]:
tokenized_datasets['train'] = tokenized_datasets['train'].remove_columns(['source','summary_target', 'tags'])
tokenized_datasets['validation'] = tokenized_datasets['validation'].remove_columns(['source','summary_target', 'tags'])
tokenized_datasets['test'] = tokenized_datasets['test'].remove_columns(['source','summary_target', 'tags'])

tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_tags'],
        num_rows: 80
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_tags'],
        num_rows: 10
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_tags'],
        num_rows: 10
    })
})

In [17]:
class MySeq2SeqTrainer(Seq2SeqTrainer):
    def prediction_step(
        self,
        model: nn.Module,
        inputs: Dict[str, Union[torch.Tensor, Any]],
        prediction_loss_only: bool,
        ignore_keys: Optional[List[str]] = None,
    ) -> Tuple[Optional[float], Optional[torch.Tensor], Optional[torch.Tensor]]:
        """
        Perform an evaluation step on `model` using `inputs`.
        Subclass and override to inject custom behavior.
        Args:
            model (`nn.Module`):
                The model to evaluate.
            inputs (`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.
                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument `labels`. Check your model's documentation for all accepted arguments.
            prediction_loss_only (`bool`):
                Whether or not to return the loss only.
        Return:
            Tuple[Optional[float], Optional[torch.Tensor], Optional[torch.Tensor]]: A tuple with the loss, logits and
            labels (each being optional).
        """
        if not self.args.predict_with_generate or prediction_loss_only:
            return super().prediction_step(
                model, inputs, prediction_loss_only=prediction_loss_only, ignore_keys=ignore_keys
            )

        has_labels = "labels" in inputs
        inputs = self._prepare_inputs(inputs)
        
        # print("prediction_step inputs: {}".format(inputs.keys()))

        # XXX: adapt synced_gpus for fairscale as well
        gen_kwargs = self._gen_kwargs.copy()
        if gen_kwargs.get("max_length") is None and gen_kwargs.get("max_new_tokens") is None:
            gen_kwargs["max_length"] = self.model.config.max_length
        
        # disable beam search
        #gen_kwargs["num_beams"] = (
        #    gen_kwargs["num_beams"] if gen_kwargs.get("num_beams") is not None else self.model.config.num_beams
        #)
        
        # enable greedy search
        gen_kwargs["num_beams"] = 1
        gen_kwargs['early_stopping'] = False
        
        # default_synced_gpus = True if is_deepspeed_zero3_enabled() else False
        default_synced_gpus = False
        gen_kwargs["synced_gpus"] = (
            gen_kwargs["synced_gpus"] if gen_kwargs.get("synced_gpus") is not None else default_synced_gpus
        )

        if "attention_mask" in inputs:
            gen_kwargs["attention_mask"] = inputs.get("attention_mask", None)
        if "global_attention_mask" in inputs:
            gen_kwargs["global_attention_mask"] = inputs.get("global_attention_mask", None)

        # prepare generation inputs
        # some encoder-decoder models can have varying encoder's and thus
        # varying model input names
        if hasattr(self.model, "encoder") and self.model.encoder.main_input_name != self.model.main_input_name:
            generation_inputs = inputs[self.model.encoder.main_input_name]
        else:
            generation_inputs = inputs[self.model.main_input_name]

        tags = inputs["decoder_tags"]
        gen_kwargs.update({"decoder_tags": tags})
        # print(f"Gen kwargs: {gen_kwargs}")
        # print(f"Gen inputs:{generation_inputs}")
         #generated_tokens = self.model.generate(
        #    generation_inputs,
        #    **gen_kwargs,
        #)
        
        gen_mix = GenerationMixin(model)
        generated_tokens, classification_ids = gen_mix.generate(generation_inputs, **gen_kwargs)
        
        dialog = tokenizer.batch_decode(generation_inputs, skip_special_tokens=True)
        print('-'*89)
        print('dialog:\n', dialog)
        
        generated_summaries = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        
        print('\n\nGenerated Summaries:\n',*generated_summaries, sep='\n')
        print(f'Generated summary length: {generated_tokens.shape}')
        
        classification_labels = []
        classification_ids_lst = classification_ids.cpu().detach().tolist()
        for batch_classification_ids in classification_ids_lst:
            batch_classification_labels = []
            for classification_id in batch_classification_ids:
                classification_id = classification_id - 3
                if classification_id >= 0 and classification_id < len(id2label):
                    batch_classification_labels.append(id2label[classification_id])
            
            classification_labels.append(' '.join(batch_classification_labels))
        
        print('\nGenerated Classification Labels:\n',*classification_labels, sep='\n')
        print(f'Generated classification tag length: {classification_ids.shape}')
        
       
        # in case the batch is shorter than max length, the output should be padded
        if gen_kwargs.get("max_length") is not None and generated_tokens.shape[-1] < gen_kwargs["max_length"]:
            generated_tokens = self._pad_tensors_to_max_len(generated_tokens, gen_kwargs["max_length"])
        elif gen_kwargs.get("max_new_tokens") is not None and generated_tokens.shape[-1] < (
            gen_kwargs["max_new_tokens"] + 1
        ):
            generated_tokens = self._pad_tensors_to_max_len(generated_tokens, gen_kwargs["max_new_tokens"] + 1)

        with torch.no_grad():
            if has_labels:
                with self.compute_loss_context_manager():
                    outputs = model(**inputs) # lm_logits as output
                if self.label_smoother is not None:
                    loss = self.label_smoother(outputs, inputs["labels"]).mean().detach()
                else:
                    loss = (outputs["loss"] if isinstance(outputs, dict) else outputs[0]).mean().detach()
            else:
                loss = None

        if self.args.prediction_loss_only:
            return (loss, None, None)

        if has_labels:
            labels = inputs["labels"]
            if gen_kwargs.get("max_length") is not None and labels.shape[-1] < gen_kwargs["max_length"]:
                labels = self._pad_tensors_to_max_len(labels, gen_kwargs["max_length"])
            elif gen_kwargs.get("max_new_tokens") is not None and labels.shape[-1] < (
                gen_kwargs["max_new_tokens"] + 1
            ):
                labels = self._pad_tensors_to_max_len(labels, (gen_kwargs["max_new_tokens"] + 1))
        else:
            labels = None
        # print(labels)

        return (loss, generated_tokens, labels)

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints_lmhead/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=4,
    num_train_epochs=5,
    predict_with_generate=True,
    do_train=True,
    do_eval=True,
    fp16=True,
    logging_steps=1,
    save_strategy="epoch",
    metric_for_best_model="f1",
    greater_is_better=True,
    load_best_model_at_end=True,
    seed=42,
    generation_max_length=max_target_length,
)

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred


    print("In compute metrics")

    print(predictions[0])

    print(labels[0])
    
    preds = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    # print(preds[0])
    flattened_preds = [item for sublist in preds for item in sublist]
    # decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # print(decoded_preds
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # print(labels[0])
    flattened_labels = [item for sublist in labels for item in sublist]

    result = f1_metric.compute(predictions=flattened_preds, references=flattened_labels, average='micro')
    # decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # print("Decoded preds and labels")

    # print(decoded_preds)
    # print(decoded_labels)
    
    # # Rouge expects a newline after each sentence
    # decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    # decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    print(result.items())
    #result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    # result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [21]:
trainer = MySeq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [22]:
model

myBartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): Laye

In [23]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: devavratj. Use `wandb login --relogin` to force relogin


You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,15.932500,11.666988,0.168800
2,21.536200,11.407684,0.168800
3,14.051700,11.530451,0.168800
4,14.947400,11.317176,0.168800
5,22.830700,11.260351,0.168800


-----------------------------------------------------------------------------------------
dialog:
 [" Adam : Have you talked to May? Karen : Yes, yesterday, why? Adam : I just talked to her and I must admit I worry about her Karen : Me too, I suggested she should see a specialist, but she wasn't very happy about it Adam : No wonder... Karen : I know, but I think this is serious. She's saying she's depressed, like everyone around, but in her case it may be true Adam : She was telling me she doesn't feel like doing anything, she's bored all the time, she never feels happy. It sounds like a real, typical depression Adam : She also told me that she has trouble sleeping. I asked her to go out for a beer or anything basically, but she doesn't want to leave the flat Karen : Oh my, it sounds really serious. I don't what to tell you Adam : I was wondering how I can help her Karen : Honestly I don't know if we can help her, Adam. I suggested a specialist because these are very sensitive issues a

-----------------------------------------------------------------------------------------
dialog:
 [' Mary : hey, im kinda broke, lend me a few box Carter : okay, give me an hour, im at the train station Mary : cool, thanks']


Generated Summaries:

Mary : hey, im at the train station. Mary : hey, im kinda broke, lend me a few box Carter : okay, give me an hour, im in the train. Mary: cool, thanks
Generated summary length: torch.Size([1, 45])

Generated Classification Labels:

O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
Generated classification tag length: torch.Size([1, 45])
-----------------------------------------------------------------------------------------
dialog:
 [' Ernest : hey Mike, did you park your car on our street? Mike : no, took it into garage today Ernest : ok good Mike : why? Ernest : someone just crashed into a red honda looking just like yours Mike : lol lucky me']


Generated Summaries:

Mike : hey Ernest, I just got a 

...

-----------------------------------------------------------------------------------------
dialog:
 [" Deirdre : Hi Beth, how are you love? Beth : Hi Auntie Deirdre, I'm been meaning to message you, had a favour to ask. Deirdre : Wondered if you had any thought about your Mum's 40th, we've got to do something special! Beth : How about a girls weekend, just mum, me, you and the girls, Kira will have to come back from Uni, of course. Deirdre : Sounds fab! Get your thinking cap on, it's only in 6 weeks! Bet she's dreading it, I remember doing that! Beth : Oh yeah, we had a surprise party for you, you nearly had a heart attack! Deirdre : Well, it was a lovely surprise! Gosh, thats nearly 4 years ago now, time flies! What was the favour, darling? Beth : Oh, it was just that I fancied trying a bit of work experience in the salon, auntie. Deirdre : Well, I am looking for Saturday girls, are you sure about it? you could do well in the exams and go on to college or 6th form. Beth : I know, but i

-----------------------------------------------------------------------------------------
dialog:
 [" Augustine : Guys, remember it's Wharton's bday next week? Darlene : yay, a party! Heather : yay! crap we need to buy him a present Walker : he mentioned paper shredder once Augustine : wtf?!? Walker : he did really. for no reason at all. Heather : whatever that make him happy Darlene : cool with me. we can shred some papers at the party Augustine : so much fun Heather : srsly guys, you mean we should really get office equipment??? Darlene : Walk, ask him if he really wnts it and if he yes then we get it Walker : i heard him say that. wasn ; t drunk. me neither. Darlene : but better ask him twice Walker : will do Augustine : 2moro ok? Darlene : and sure ask ab the party!"]


Generated Summaries:

Augustine : Hey guys, we need to talk about something. Augustine : Guys, remember it's Wharton's bday next week? Darlene : yay, a party! Heather : yacch, we should go to the party! Augustine : 

-----------------------------------------------------------------------------------------
dialog:
 [" Adam : Have you talked to May? Karen : Yes, yesterday, why? Adam : I just talked to her and I must admit I worry about her Karen : Me too, I suggested she should see a specialist, but she wasn't very happy about it Adam : No wonder... Karen : I know, but I think this is serious. She's saying she's depressed, like everyone around, but in her case it may be true Adam : She was telling me she doesn't feel like doing anything, she's bored all the time, she never feels happy. It sounds like a real, typical depression Adam : She also told me that she has trouble sleeping. I asked her to go out for a beer or anything basically, but she doesn't want to leave the flat Karen : Oh my, it sounds really serious. I don't what to tell you Adam : I was wondering how I can help her Karen : Honestly I don't know if we can help her, Adam. I suggested a specialist because these are very sensitive issues a

-----------------------------------------------------------------------------------------
dialog:
 [' Mary : hey, im kinda broke, lend me a few box Carter : okay, give me an hour, im at the train station Mary : cool, thanks']


Generated Summaries:

Mary : hey, im at the train station. Mary : hey, im kinda broke, lend me a few box Carter : okay, give me an hour, im in the train. Mary: cool, thanks
Generated summary length: torch.Size([1, 45])

Generated Classification Labels:

O O O O O O O O O O O O O O O O O O O O C O O O O O O O O O O O O O O O O O O O O O O O
Generated classification tag length: torch.Size([1, 45])
-----------------------------------------------------------------------------------------
dialog:
 [' Ernest : hey Mike, did you park your car on our street? Mike : no, took it into garage today Ernest : ok good Mike : why? Ernest : someone just crashed into a red honda looking just like yours Mike : lol lucky me']


Generated Summaries:

Mike : hey Ernest, I just got a 

TrainOutput(global_step=400, training_loss=15.311861289739609, metrics={'train_runtime': 304.434, 'train_samples_per_second': 1.314, 'train_steps_per_second': 1.314, 'total_flos': 216988411084800.0, 'train_loss': 15.311861289739609, 'epoch': 5.0})

In [24]:
trainer.evaluate(tokenized_datasets['test'])

-----------------------------------------------------------------------------------------
dialog:
 [' Mary : hey, im kinda broke, lend me a few box Carter : okay, give me an hour, im at the train station Mary : cool, thanks']


Generated Summaries:

Mary : hey, im at the train station. Mary : hey, im kinda broke, lend me a few box Carter : okay, give me an hour, im in the train. Mary: cool, thanks
Generated summary length: torch.Size([1, 45])

Generated Classification Labels:

O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
Generated classification tag length: torch.Size([1, 45])


-----------------------------------------------------------------------------------------
dialog:
 [" Jesse : I have an idea that'll cheer u up! Melvin : What is it? Jesse : I was thinking about doing something 4 the less fortunate this year. Lee : Gr8 idea! Anything in mind? Maxine : So no presents 4 me? : ( Jesse : U'll get ur presents, no worries ; ) Maxine : Phew! Was getting a bit worried for a moment ; ) Melvin : Bt what do u have in store? Jesse : Well, have u heard about the Refuge? Lee : No. What's that? Melvin : That's the Christmas foundation to help women and children? Maxine : I think I've heard of them. So what about them? Jesse : That's right! They help women and children who escape from abuse. And every year they post wish lists of such ppl online and I thought that we could choose one and chip in. Melvin : That's a great idea! Lee : Count me in! Maxine : Me too. Jesse : Have a look at these 3 lists : <file_other> <file_other> <file_other> Lee : I think the second one w

-----------------------------------------------------------------------------------------
dialog:
 [" Gloria : This exam is a bit of a lottery in fact Gloria : You can't really get prepared, it's all about experience Emma : But there are some rules and some typical texts right? Gloria : You can see some texts from previous years Gloria : <file_other> Emma : Wow that's very useful Emma : I have never seen this site Gloria : Yes it's very good Gloria : Actually it's good to read all the texts because you will see that some phrases repeat very often Emma : How much time do you have for all 4 parts? Gloria : 4 hours Emma : Is it enough? Gloria : Well it has to be Gloria : Would be perfect to have 2 more hours... But on the other hand it would be really exhausting Emma : 4 hours and no breaks? Gloria : No breaks : / So it's really important to be really focused and try to write as fast as you can Gloria : And read it carefully and correct during the last hour Emma : I'm going to read everyt

{'eval_loss': 15.563984870910645,
 'eval_f1': 0.1023,
 'eval_runtime': 43.1704,
 'eval_samples_per_second': 0.232,
 'eval_steps_per_second': 0.232,
 'epoch': 5.0}

In [25]:
model.save_pretrained("hallucination-tagging-classifier-lmhead")

In [26]:
torch.cuda.empty_cache()

In [27]:
!nvidia-smi

Sun Dec  3 21:05:28 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.13                 Driver Version: 537.13       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 Ti   WDDM  | 00000000:01:00.0 Off |                  N/A |
|  0%   41C    P8              15W / 310W |   7545MiB /  8192MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--